In [25]:
import torch
from evogp.tree import Forest, GenerateDescriptor
from evogp.algorithm import (
    GeneticProgramming,
    DefaultSelection,
    DefaultMutation,
    DefaultCrossover,
)
from evogp.problem import SymbolicRegression
from evogp.pipeline import StandardPipeline

XOR_INPUTS = torch.tensor(
    [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
    ],
    dtype=torch.float,
    device="cuda",
)

XOR_OUTPUTS = torch.tensor(
    [[0], [1], [1], [0], [1], [0], [0], [1]],
    dtype=torch.float,
    device="cuda",
)

problem = SymbolicRegression(datapoints=XOR_INPUTS, labels=XOR_OUTPUTS)

# create decriptor for generating new trees
descriptor = GenerateDescriptor(
    max_tree_len=128,
    input_len=problem.problem_dim,
    output_len=problem.solution_dim,
    using_funcs=["+", "-", "log", "sqrt", "pow", "/", "inv"],
    max_layer_cnt=2,
    const_samples=[-1, 0, 1],
)

# create the algorithm
algorithm = GeneticProgramming(
    initial_forest=Forest.random_generate(pop_size=100000, descriptor=descriptor),
    crossover=DefaultCrossover(),
    mutation=DefaultMutation(
        mutation_rate=0.2, descriptor=descriptor
    ),
    selection=DefaultSelection(survival_rate=0.3, elite_rate=0.01),
)

pipeline = StandardPipeline(
    algorithm,
    problem,
    generation_limit=1000,
    fitness_target = -0.001,
    # is_show_details=False,
)

best = pipeline.run()
best_fitness = best.SR_fitness(XOR_INPUTS, XOR_OUTPUTS)
print(best_fitness)

pred_res = best.forward(XOR_INPUTS)
print(pred_res)

sympy_expression = best.to_sympy_expr()
print(sympy_expression)

# best.to_png("./imgs/xor_tree.png")

Generation: 0, Cost time: 19.38ms
 	fitness: valid cnt: 64994, max: -0.5000, min: -6.5000, mean: -1.0122, std: 0.9593

Generation: 1, Cost time: 21.00ms
 	fitness: valid cnt: 87216, max: -0.2500, min: -12.5000, mean: -0.6616, std: 0.5671

Generation: 2, Cost time: 17.00ms
 	fitness: valid cnt: 88649, max: -0.2500, min: -12.5000, mean: -0.6177, std: 0.4831

Generation: 3, Cost time: 15.00ms
 	fitness: valid cnt: 89576, max: -0.2500, min: -12.5000, mean: -0.6101, std: 0.4739

Generation: 4, Cost time: 14.22ms
 	fitness: valid cnt: 89604, max: -0.2500, min: -12.5000, mean: -0.5963, std: 0.4390

Generation: 5, Cost time: 15.00ms
 	fitness: valid cnt: 89292, max: -0.2500, min: -9.5000, mean: -0.5975, std: 0.4564

Generation: 6, Cost time: 15.00ms
 	fitness: valid cnt: 88378, max: -0.2500, min: -35.4968, mean: -0.6069, std: 0.5067

Generation: 7, Cost time: 20.00ms
 	fitness: valid cnt: 86898, max: -0.2500, min: -36.4071, mean: -0.6207, std: 0.5435

Generation: 8, Cost time: 15.00ms
 	fitnes

In [26]:
sympy_expression

((1.0 + 1/(x0 + x1 + x2 - 1.0))**(-0.5) + 1.0)**(-sqrt(1.0*x1**1.0 + 1/(-sqrt(x0) - x1 - x2 + 2.0)))

In [13]:
sympy_expression.simplify()

0.5**(0.0**((x0 - 0.5)*(x1 - 0.5)*(x2 - 0.5)))

In [14]:
best.to_infix()

'pow(0.50,pow(pow(0.00,(x[0] − 0.50)),((x[1] − 0.50) * (x[2] − 0.50))))'